In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import pearsonr

from quant_invest_lab.data_provider import download_crypto_historical_data

In [2]:
symbol= 'BTC-USDT'
timeframe = "1hour"
df_BTC = download_crypto_historical_data(symbol, timeframe).iloc[-3000:]
df_BTC["Returns"] = df_BTC.Close.pct_change()
df_BTC.dropna(inplace=True)
print(df_BTC.shape)
df_BTC.head()

(2999, 8)


,Timestamp,Open,Close,High,Low,Amount,Volume,Returns
Date,,,,,,,,
2023-01-13 16:00:00,1673622000,18998.9,19260.5,19309.5,18993.4,1160.606223,2.224767e+07,0.013769
2023-01-13 17:00:00,1673625600,19260.6,19131.2,19271.6,19064.0,427.925713,8.189067e+06,-0.006713
2023-01-13 18:00:00,1673629200,19131.2,19256.2,19392.2,19106.7,485.314231,9.359598e+06,0.006534
2023-01-13 19:00:00,1673632800,19256.2,19292.8,19384.4,19256.1,249.704990,4.822401e+06,0.001901
2023-01-13 20:00:00,1673636400,19292.8,19360.6,19369.0,19247.7,222.083237,4.286924e+06,0.003514


In [3]:
df_BTC['EMA20'] = df_BTC.Close.ewm(20).mean()
df_BTC['EMA100'] = df_BTC.Close.ewm(100).mean()
df_BTC = df_BTC.drop(columns=["Timestamp"])

df_BTC

,Open,Close,High,Low,Amount,Volume,Returns,EMA20,EMA100
Date,,,,,,,,,
2023-01-13 16:00:00,18998.9,19260.5,19309.5,18993.4,1160.606223,2.224767e+07,0.013769,19260.500000,19260.500000
2023-01-13 17:00:00,19260.6,19131.2,19271.6,19064.0,427.925713,8.189067e+06,-0.006713,19194.273171,19195.528358
2023-01-13 18:00:00,19131.2,19256.2,19392.2,19106.7,485.314231,9.359598e+06,0.006534,19215.930373,19215.953804
2023-01-13 19:00:00,19256.2,19292.8,19384.4,19256.1,249.704990,4.822401e+06,0.001901,19236.576225,19235.453038
2023-01-13 20:00:00,19292.8,19360.6,19369.0,19247.7,222.083237,4.286924e+06,0.003514,19263.858478,19260.982994
...,...,...,...,...,...,...,...,...,...
2023-05-18 11:00:00,27389.1,27406.2,27430.1,27354.8,80.118878,2.194195e+06,0.000628,27166.907749,27260.738684
2023-05-18 12:00:00,27406.2,27347.2,27424.8,27331.9,72.185960,1.976658e+06,-0.002153,27175.493094,27261.594737
2023-05-18 13:00:00,27347.3,27395.0,27396.8,27343.9,80.362286,2.200075e+06,0.001748,27185.945804,27262.915581


# Predict crypto from BTC

In [4]:
symbol_to_predict= 'AVAX-USDT'

df_to_predict= download_crypto_historical_data(symbol_to_predict, timeframe).iloc[-3000:]

df_to_predict["Return"] = df_to_predict.Close.pct_change()
df_to_predict.dropna(inplace=True)
print(df_to_predict.shape)
df_to_predict.head()

(2999, 9)


,Timestamp,Open,Close,High,Low,Amount,Volume,Returns,Return
Date,,,,,,,,,
2023-01-13 16:00:00,1673622000,15.548,15.655,15.776,15.523,31755.2328,497430.490955,0.006817,0.006817
2023-01-13 17:00:00,1673625600,15.654,15.434,15.658,15.357,26782.1100,414346.256304,-0.014117,-0.014117
2023-01-13 18:00:00,1673629200,15.426,15.541,15.596,15.373,19565.1474,302739.940217,0.006933,0.006933
2023-01-13 19:00:00,1673632800,15.544,15.515,15.607,15.499,19717.8677,306748.336637,-0.001673,-0.001673
2023-01-13 20:00:00,1673636400,15.522,15.640,15.699,15.492,25510.3952,397892.497186,0.008057,0.008057


In [5]:
merged = pd.merge(df_BTC, df_to_predict, left_index=True,right_index=True,suffixes=('',f'_{symbol_to_predict}'))[['Close',f'Close_{symbol_to_predict}']]

merged.head()

,Close,Close_AVAX-USDT
Date,,
2023-01-13 16:00:00,19260.5,15.655
2023-01-13 17:00:00,19131.2,15.434
2023-01-13 18:00:00,19256.2,15.541
2023-01-13 19:00:00,19292.8,15.515
2023-01-13 20:00:00,19360.6,15.640


In [8]:
SHIFT = 100
fig = make_subplots(
    rows=1, cols=1
)

fig.add_trace(
    go.Scatter(
        name="Not shifted",
        mode='markers',
        x=merged["Close"],
        y=merged[f'Close_{symbol_to_predict}'],
        opacity=0.5,
        marker=dict(
            color='Red',
            size=7,
        ),
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        name="Shifted",
        mode='markers',
        x=merged["Close"].iloc[:-SHIFT],
        y=merged[f'Close_{symbol_to_predict}'].iloc[SHIFT:],
        opacity=0.5,
        marker=dict(
            color='blue',
            size=7,
        ),
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Comparing relation between pair to pair data and shifted data",
)



In [9]:
print(pearsonr(merged["Close"].iloc[:-SHIFT],merged[f'Close_{symbol_to_predict}'].iloc[SHIFT:]))

PearsonRResult(statistic=-0.09934125624912939, pvalue=8.33495705621818e-08)
